### Попробуем обучить модель проверять свободные ответы школьников на тестовые задания

Будем решать задачу с помощью семантических векторов и линейного классификатора.
Пример тестового задания есть на картинке в репозитории.

In [ ]:
# Для начала необходимо обновить установщик пакетов pip
!pip install --upgrade pip

In [ ]:
# Устанавливаем необходимые библиотеки
!pip install tensorflow-hub==0.16.1, tensorflow==2.18.1, tensorflow-text==2.18.1, psycopg2-binary

Теперь необходимо перезагрузить kernel, чтобы JupyterHub увидел все изменения в составе библиотек. Нажмите круговую стрелку в верхнем меню этого ноутбука или зайдите в меню Kernel -> Restart Kernel and Clear Outputs of All Cells... Так ноутбук будет выглядеть опрятнее.

In [ ]:
# Загружаем необходимые библиотеки
import mlflow
import mlflow.sklearn

import numpy as np
import pandas as pd
import psycopg2

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as prfs

In [ ]:
# Загружаем модель Universal Sentence Encoder для векторизации текстов
embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/multilingual/2")

In [ ]:
# Подключаемся к БД, используя ее внутренний ip, и вычитываем данные из таблицы в переменную
connection = psycopg2.connect(database="kd_pg_nu", user="admin", password="732n0j2P(hD4dB12n", host="10.0.0.137", port=5432)
cursor = connection.cursor()
cursor.execute("SELECT * from public.dataset;")
dataset = cursor.fetchall()

In [ ]:
# Посмотрим на данные
dataset[:5]

In [ ]:
# Преобразуем датасет в формат pandas DataFrame, чтобы с ним было удобнее работать
text = []
mark = []
ID = []

for row in dataset:
    text.append(row[0])
    mark.append(row[1])
    ID.append(row[2])

data = {'text': text, 'mark': mark, 'id': ID}
data = pd.DataFrame(data)

In [ ]:
# Посмотрим на данные
data.head() 

In [ ]:
# Запишем названия нужных колонок в переменные, это пригодится дальше
text_var = 'text'
class_var = 'mark'

In [ ]:
# Посмотрим на распреление оценок в датасете
df = data[class_var].value_counts(normalize=True) * 100
df.plot.bar(x=class_var)

В данном задании оценка ставится по трехбалльной шкале:
- 0 баллов за неправильный ответ
- 1 балл за правильный ответ
- 2 балла за правильный ответ и правильное объяснение своего ответа

Оценка -1 ставится за отсутствие ответа, и бессмысленный набор символов. Такие случаи проще ловить отдельным кодом и не использовать их для обучения модели.

In [ ]:
# Исключаем из датасета ответы с оценкой -1
data = data[(data[class_var] != -1)]

В датасете осталось три оценки: 0,1 и 2 балла. Для нас это значит, что надо решить задачу классификации на 3 класса.

Эту задачу можно решать напрямую, а можно поэтапно. Ответы на 1 балл и на 2 балла очень похожи, поэтому будет сложно обучить модель сразу оценивать ответы по трехбалльной шкале. Давайте двигаться поэтапно и сначала научим модель отличать точно неправильные ответы от сколько-нибудь правильных.

In [ ]:
# Превращаем оценку 2 в 1, чтобы оставить только два класса
marks_new = []
for i, mark in enumerate(data[class_var]):
    if mark == 2:
        marks_new.append(1)
    else:
        marks_new.append(mark)
class_var = 'class_new'
data[class_var] = marks_new
del marks_new

До этого мы занимались извлечением и подготовкой данных, то есть можно сказать занимались дата-инжинирингом. Дальше начинается раздел дата-сайенс, мы будем заниматься машинным обучением.

Нельзя просто взять и обучить модель. Нужно подобрать тип модели и нужные параметры к ней. Это всегда множество экспериментов, результаты которых нужно сохранять, чтобы потом выбрать лучший. За это отвечает модуль MLFlow, который в следующей ячейке мы и запустим.

In [ ]:
# Запускаем логирование эксперимента
mlflow.set_experiment("Auto_logging")
mlflow.sklearn.autolog(log_input_examples=True)
mlflow.start_run()

In [ ]:
# Устанавливаем размер обучающей выборки в процентах
train_vol = 0.7

In [ ]:
# Делим датасет на обучающую и тестовую выборки. Из обучающей выборки удаляем дубликаты ответов
train, test = train_test_split(data, train_size = train_vol, random_state = 99, stratify = data[class_var])
train = train.drop_duplicates(subset=[text_var])

In [ ]:
# Из тестовой выборки удаляем все ответы, которые есть в обучающей
check = []
train_data = train[text_var].tolist()
for text in test[text_var]:
    if text not in train_data:
        check.append('KEEP')
    else:
        check.append('IGNORE')
test['check'] = check
test = test.loc[test['check'] == 'KEEP']
test = test.drop(['check'], axis=1)
del check

In [ ]:
# Переводим все ответы из обучающей выборки в семантические вектора
train_embeddings = []
for text in train_data:
    embedding = embed(str(text))[0]
    train_embeddings.append(embedding)

train_embeddings = np.array(train_embeddings)

In [ ]:
# Переводим все ответы из тестовой выборки в вектора
test_embeddings = []
for text in test[text_var]:
    embedding = embed(str(text))[0]
    test_embeddings.append(embedding)

test_embeddings = np.array(test_embeddings)

In [ ]:
# Инициализируем классификатор с некоторыми параметрами
svm = SVC(kernel='poly', gamma='scale', probability=True)

In [ ]:
# Получаем список оценок из обучающей и тестовой выборок
classes = np.array(train[class_var])
classes_to_check = np.array(test[class_var])

In [ ]:
# Обучаем классификатор
svm.fit(train_embeddings,classes)

In [ ]:
# Применяем модель к тестовой выборке
classesSVM = svm.predict(test_embeddings)

In [ ]:
# Получаем точность, полноту и ф-меру для класса 0 (неправильные ответы)
metrics = prfs(classes_to_check, classesSVM, pos_label = 0, average = 'binary')

In [ ]:
print('Precision: ', metrics[0])
print('Recall: ', metrics[1])
print('F-Score: ', metrics[2])

Видно, что модель научилась очень качественно определять неправильные ответы. Обратите внимание, для этого не нужны LLM и GPU. Можно закончить эксперимент и залогировать результаты.

In [ ]:
# Заканчиваем эксперимент, логируем результаты и регистрируем версию модели
mlflow.sklearn.log_model(svm, "model", registered_model_name="Test-evaluation")
mlflow.end_run()

На этом закончилась часть дата-сайенс, мы обучили модель и сохранили необходимые артефакты. Далее мы попробуем себя в роли DevOps-инженеров и развернем нашу обученную модель как сервис, доступный для внешних пользователей.

In [ ]:
# Создаем экземпляр класса MlflowClient. Это необходимый компонент для развертывания сервиса
from mlflow.tracking import MlflowClient
cli = MlflowClient()

In [ ]:
# Берем для деплоя самую первую (и пока единственную) зарегистрированную модель
model_source_uri = cli.search_registered_models()[0].latest_versions[0].source
print("Имя модели: ", cli.search_registered_models()[0].latest_versions[0].name)
print("URI модели: ", model_source_uri)

In [ ]:
# Так можно получить список всех моделей и вывести их с сортировкой по имени
results = cli.search_registered_models(order_by=["name ASC"])
print("-" * 80)
for res in results:
    for mv in res.latest_versions:
        print("name={}; run_id={}; version={}; source={};".format(mv.name, mv.run_id, mv.version, mv.source))

In [ ]:
# Создаем экземпляр класса Client для работы в VK Cloud ML Platform
from mlflow.deployments import get_deploy_client
client = get_deploy_client('vk-cloud-mlplatform')

endpoint в терминологии VK Cloud MLflow Deploy - это деплой сервер. Отдельная машина на которой непосредственно разворачивается контейнер с моделью.

In [ ]:
# Создаем эндпоинт. Создание обычно занимает 5-10 минут
deploy_server_name = "deploy_server"
client.create_endpoint(name=deploy_server_name)

In [ ]:
# Получаем информацию о статусе деплой сервера по его имени
client.get_endpoint(deploy_server_name)

In [ ]:
# Удаляем деплой сервер, если необходимо. Пока пропускаем эту ячейку
#client.delete_endpoint(deploy_server_name)

In [ ]:
# Создаем deployment - запущеный докер с моделью на сервере. Создание занимает примерно 10 минут
deployment_name="test_deployment"
client.create_deployment(server_name=deploy_server_name, deployment_name=deployment_name, model_uri=model_source_uri)

In [ ]:
# Получаем статус и информацию о задеплоенной модели по имени деплой сервера и модели
client.get_deployment(deploy_server_name, deployment_name)

In [ ]:
# Удаляем deployment с сервера, обращаясь по имени сервера и deployment. Пока пропускаем эту ячейку
#client.delete_deployment(deploy_server_name, deployment_name)

Наша модель развернута на сервере MLFlow Deploy и готова к работе, ура! Давайте попробуем обратиться к ней. Для этого напишем один правильный ответ и один неправильный ответ на тестовое задание, получим вектора этих ответов и отправим на оценку модели.

In [ ]:
wrong = 'Движение машины по дороге, потому что она едет быстро'
right = 'Скисание молока в теплой комнате, потому что происходит превращение вещества'

inputs = [embed(wrong)[0].numpy().tolist(), embed(right)[0].numpy().tolist()]

In [ ]:
# Вызываем метод predict для проверки доступности модели
data = {"inputs": inputs}
client.predict(deploy_server_name, deployment_name, data)

Также можно создать deployment, который будет доступен по публичному DNS имени, то есть к которому можно обращаться из вне облака. Например с вашей локальной машины. Для доступа к модели нужно будет указать логин и пароль, то есть внешний пользователь не сможет делать запросы к модели без авторизации

Обязательно замените имя пользователя и пароль на новые. Имя пользователя и пароль рекомендуется задавать отличное от логина/пароля для входа в облако или в JupyterHub.

In [ ]:
auth_value = "user:PasswordDA@dvv//!123$"
auth_deployment_name = "test_deploy_auth"
client.create_deployment(deploy_server_name, auth_deployment_name, model_source_uri, auth=auth_value)

In [ ]:
# Получаем статус и информацию о задеплоенной модели
deployment_info = client.get_deployment(deploy_server_name, auth_deployment_name)
print(deployment_info)

# Получаем DNS имя для обращения к модели
print(deployment_info['model_ref'])

In [ ]:
# Отправляем запрос с помощью requests. 
# Обратите внимание, в запросе нужно передать данные для авторизации

import requests
data = {"inputs": inputs}
response = requests.post('', json=data, auth=("user", "PasswordDA@dvv//!123$"))

print(response.text)